# Aula 6 - Modelos Matemáticos no Pyomo


### Instalando as bibliotecas 

https://pyomo.readthedocs.io/en/stable/

In [1]:
pip install pyomo

     |████████████████████████████████| 8.9MB 2.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 


In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Setting up libcolamd2:amd64 (1:5.1.2-2) ...
Setting up libamd2:amd64 

### Importando as bibliotecas

In [3]:
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels

### Importando o dataframe a ser utilizado

In [5]:
df = pd.read_csv('/content/drive/Shareddrives/Disciplina Modelagem/Aulas/Aula 3/cvrp_RJ0_toy1_instancia_cluster4', index_col = 0)

,rotas,demandas,latitude,longitude,Cluster
0,origem,NaN,-22.747450,-42.881871,4
1,"{'lng': -42.92705867402125, 'lat': -22.7881124...",7.0,-22.788112,-42.927059,4
2,"{'lng': -42.827461886756275, 'lat': -22.725373...",10.0,-22.725374,-42.827462,4
3,"{'lng': -42.89710859232077, 'lat': -22.7893503...",5.0,-22.789350,-42.897109,4
4,"{'lng': -42.88723394420894, 'lat': -22.7463572...",1.0,-22.746357,-42.887234,4
5,"{'lng': -42.93574292860143, 'lat': -22.7390110...",3.0,-22.739011,-42.935743,4
6,"{'lng': -42.87052927395793, 'lat': -22.7445850...",9.0,-22.744585,-42.870529,4
7,"{'lng': -42.8096570758958, 'lat': -22.73526798...",9.0,-22.735268,-42.809657,4
8,"{'lng': -42.85149055565491, 'lat': -22.7519213...",10.0,-22.751921,-42.851491,4
9,"{'lng': -42.91694664841375, 'lat': -22.7558571...",1.0,-22.755857,-42.916947,4


In [ ]:
from math import radians
df['lat_rad'] = df['latitude'].apply(radians)
df['long_rad'] = df['longitude'].apply(radians)

In [ ]:
# importando o pacote de distância e escolhendo a distância
from sklearn.neighbors import DistanceMetric
dist = DistanceMetric.get_metric('haversine')

In [ ]:
# precisamos transformar para array
df[['lat_rad','long_rad']].to_numpy()

array([[-0.39701789, -0.74842984],
       [-0.39772759, -0.74921851],
       [-0.3966326 , -0.74748022],
       [-0.3977492 , -0.74869578],
       [-0.39699883, -0.74852344],
       [-0.39687061, -0.74937008],
       [-0.3969679 , -0.74823189],
       [-0.39680528, -0.74716947],
       [-0.39709594, -0.7478996 ],
       [-0.39716463, -0.74904202],
       [-0.3968901 , -0.74795948],
       [-0.39712629, -0.74848462],
       [-0.39704211, -0.74818184],
       [-0.39730161, -0.74798599],
       [-0.39673632, -0.74941432],
       [-0.39720158, -0.74910572],
       [-0.39701989, -0.74837839],
       [-0.39702738, -0.74845472],
       [-0.39739695, -0.74938337],
       [-0.39688458, -0.74789934],
       [-0.39676161, -0.74782715],
       [-0.39647792, -0.74950425],
       [-0.39737148, -0.74957958],
       [-0.39711109, -0.74837049],
       [-0.39706339, -0.7478235 ],
       [-0.39749032, -0.74912913],
       [-0.39699884, -0.74789917],
       [-0.39672407, -0.74796093],
       [-0.39705731,

In [ ]:
# distância euclidiana - multiplicar por 6373 para passar para Km
dist.pairwise(df[['lat_rad','long_rad']].to_numpy())*6373

array([[ 0.        ,  6.47582945,  6.09786043, ...,  0.1557263 ,
         4.60322007,  4.28538414],
       [ 6.47582945,  0.        , 12.37174387, ...,  6.55802415,
        10.48611299, 10.27277669],
       [ 6.09786043, 12.37174387,  0.        , ...,  6.07087633,
         2.20870309,  5.13043419],
       ...,
       [ 0.1557263 ,  6.55802415,  6.07087633, ...,  0.        ,
         4.62466363,  4.14281857],
       [ 4.60322007, 10.48611299,  2.20870309, ...,  4.62466363,
         0.        ,  5.33387573],
       [ 4.28538414, 10.27277669,  5.13043419, ...,  4.14281857,
         5.33387573,  0.        ]])

In [ ]:
## Finalmente
distancia1 = pd.DataFrame(dist.pairwise(df[['lat_rad','long_rad']].to_numpy())*6373)

In [ ]:
dist = distancia1.iloc[0,:].tolist()

In [ ]:
df.head()

,rotas,demandas,latitude,longitude,Cluster,lat_rad,long_rad
0,origem,NaN,-22.747450,-42.881871,4,-0.397018,-0.748430
1,"{'lng': -42.92705867402125, 'lat': -22.7881124...",7.0,-22.788112,-42.927059,4,-0.397728,-0.749219
2,"{'lng': -42.827461886756275, 'lat': -22.725373...",10.0,-22.725374,-42.827462,4,-0.396633,-0.747480
3,"{'lng': -42.89710859232077, 'lat': -22.7893503...",5.0,-22.789350,-42.897109,4,-0.397749,-0.748696
4,"{'lng': -42.88723394420894, 'lat': -22.7463572...",1.0,-22.746357,-42.887234,4,-0.396999,-0.748523


In [ ]:
p_dem = df.loc[1:,'demandas'].tolist() 

In [ ]:
p_dist = (1/(distancia1.iloc[0,1:])).tolist()

In [ ]:
p_dem_dist = (df.loc[1:,'demandas']/distancia1.iloc[0,1:]).tolist()

In [ ]:
cap_vel = 13

## Problema da Mochila

Problemas da mochila envolvem a escolha de itens a serem colocados em uma ou mais mochilas de forma a maximizar uma função de utilidade. 

Neste caso, vamos considerar que a demanda seja alocada a um veículo de modo que possamos maximizar três ideias:

1. $Demanda$

2. $\frac{1}{Distancia}$


3. $\frac{Demanda}{Distancia}$


### Modelo Matemático

**Conjunto da demanda:**

$i \in D$

<br>

**Parâmetros:**

$ dem_i \rightarrow$ demanda do cliente $i$ 

$ cap \rightarrow$ capacidade do veículo 

<br>

**Variáveis de decisão:**

$ x_i \rightarrow \{0,1\} $

<br>

**Função objetivo:**

$ Max \displaystyle\sum_{i \in D} dem_ix_i$

<br>

Sujeito a:

$\displaystyle\sum_{i \in D} dem_ix_i \leq cap$


## Problema das Múltiplas Mochilas

Considere $m$ itens que devem ser colocados $m$ mochilas de capacidades distintas $b_i$, $ i = 1, ... , m $. 

In [ ]:
# vamos criar uma lista de veículos com suas capacidades
vcl = [13, 20, 30]

### Modelo 2

**Conjuntos**

- Capacidade dos veículos:   $i \in V$

- Demanda dos nós:   $j \in D$

<br>

**Parâmetros:**

$ dem_j \rightarrow$ demanda do cliente $j$ 

$ cap_i \rightarrow$ capacidade do veículo 

<br>

**Variáveis de decisão:**

$ x_{ij} \rightarrow \{0,1\} $

<br>

**Função objetivo:**

$ Max \displaystyle\sum_{i \in V}\displaystyle\sum_{j \in D} dem_jx_{ij}$

<br>

Sujeito a:

$\displaystyle\sum_{j \in D} dem_jx_{ij} \leq cap_i \quad\forall \:i \in V $

$\displaystyle\sum_{i \in V} x_{ij} \leq 1 \quad\forall \:j \in D $

## Problema Empacotamento de mochilas (bin packing)


### Modelo 3 

**Conjuntos**

- Capacidade dos veículos:   $i \in V$

- Demanda dos nós:   $j \in D$

<br>


**Parâmetros:**

$ dem_j \rightarrow$ demanda do cliente $i$ 

$ cap_i \rightarrow$ capacidade do veículo 

<br>

**Variáveis de decisão:**

$ x_{ij} \rightarrow \{0,1\} $

$ y_{i} \rightarrow \{0,1\} $

<br>

**Função objetivo:**

$Min\displaystyle\sum_{i \in V}y_{i}$

<br>

Sujeito a:

$\displaystyle\sum_{i \in V} x_{ij} = 1 \quad\forall \:j \in D $

$\displaystyle\sum_{j \in D} dem_jx_{ij} \leq cap_iy_i \quad\forall \:i \in V $

